

 # 使用 `torchtnt` 统计模型的FLOPS


 本示例展示如何使用 ``torchtnt`` 库估算模型前向传播和反向传播过程中 每秒浮点运算次数（FLOPS）。FLOPS是衡量模型计算复杂度的核心指标，反映模型的算力需求。


 本教程演示如何分析神经算子（Neural Operator）模型的计算开销，FLOPS统计的核心价值在于：

 - 对比不同模型架构的计算效率

 - 定位模型中的计算瓶颈模块

 - 优化模型的推理/训练效率

 - 为模型部署（如选择硬件）提供决策依据



 我们将通过FLOPS计算来分析FNO（Fourier Neural Operator）模型的计算资源消耗。



 ## 导入依赖库

 导入FLOPS统计和模型创建所需的核心模块



In [1]:
# 导入深拷贝工具，用于复制FLOPS统计结果（避免引用传递）
from copy import deepcopy
# 导入PyTorch核心库
import torch
# 导入torchtnt中用于统计FLOPS的核心类：基于Tensor Dispatch Mode的FLOPS计数器
from torchtnt.utils.flops import FlopTensorDispatchMode

# 从neuralop库导入FNO模型（傅里叶神经算子，常用于偏微分方程求解等科学计算场景）
from neuralop.models import FNO

# 设置计算设备为CPU（也可改为"cuda"使用GPU，需确保有可用GPU）
device = "cpu"


c:\Users\MR\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchtnt\utils\version.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
W0207 15:51:00.556000 14828 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


尝试打开的路径是: c:\Users\MR\AppData\Local\Programs\Python\Python312\Lib\site-packages
尝试打开的路径是: c:\Users\MR\AppData\Local\Programs\Python\Python312\Lib\site-packages




 ## 创建用于分析的FNO模型

 构建一个中等规模的FNO模型，用于演示FLOPS统计流程





In [2]:
# 初始化FNO模型
fno = FNO(
    n_modes=(64, 64),          # 傅里叶模态数，决定频域特征提取的维度
    in_channels=1,             # 输入通道数（示例中为单通道）
    out_channels=1,            # 输出通道数（示例中为单通道）
    hidden_channels=64,        # 隐藏层通道数，控制模型宽度
    projection_channel_ratio=1,# 投影通道比例，用于调整特征投影的维度
)

# 创建用于FLOPS统计的示例输入张量（模拟真实输入）
batch_size = 4                # 批量大小
# 生成形状为 [batch_size, in_channels, 128, 128] 的随机输入张量
model_input = torch.randn(batch_size, 1, 128, 128)






 ## 统计前向传播和反向传播的FLOPS

 使用FlopTensorDispatchMode上下文管理器，自动统计模型前向/反向传播的FLOPS

 （DispatchMode是PyTorch 2.0+的特性，可拦截张量操作并统计计算量）



In [3]:
# 启动FLOPS统计上下文管理器，传入要分析的模型
with FlopTensorDispatchMode(fno) as ftdm:
    # 执行模型前向传播，并对输出取均值（为反向传播提供标量损失）
    res = fno(model_input).mean()
    # 深拷贝前向传播的FLOPS统计结果（避免后续reset覆盖）
    fno_forward_flops = deepcopy(ftdm.flop_counts)

    # 重置FLOPS计数器（清空前向传播的统计结果）
    ftdm.reset()
    # 执行反向传播（基于前向的均值结果计算梯度）
    res.backward()
    # 深拷贝反向传播的FLOPS统计结果
    fno_backward_flops = deepcopy(ftdm.flop_counts)




 ## 分析FLOPS的详细分布

 统计结果以defaultdict嵌套字典的形式存储，键为模型子模块名称，值为对应模块的FLOPS

 这种结构可以清晰看到模型各部分的计算开销，定位计算瓶颈。





In [4]:
# 打印前向传播的FLOPS分布（按模型子模块拆分）
print("Forward pass FLOPS breakdown:")
print(fno_forward_flops)


Forward pass FLOPS breakdown:
defaultdict(<function FlopTensorDispatchMode.__init__.<locals>.<lambda> at 0x000001C873246DE0>, {'': defaultdict(<class 'int'>, {'convolution.default': 2982150144, 'bmm.default': 138412032}), 'lifting': defaultdict(<class 'int'>, {'convolution.default': 562036736}), 'lifting.fcs.0': defaultdict(<class 'int'>, {'convolution.default': 25165824}), 'lifting.fcs.1': defaultdict(<class 'int'>, {'convolution.default': 536870912}), 'fno_blocks': defaultdict(<class 'int'>, {'convolution.default': 2147483648, 'bmm.default': 138412032}), 'fno_blocks.fno_skips.0': defaultdict(<class 'int'>, {'convolution.default': 268435456}), 'fno_blocks.fno_skips.0.conv': defaultdict(<class 'int'>, {'convolution.default': 268435456}), 'fno_blocks.convs.0': defaultdict(<class 'int'>, {'bmm.default': 34603008}), 'fno_blocks.channel_mlp.0': defaultdict(<class 'int'>, {'convolution.default': 268435456}), 'fno_blocks.channel_mlp.0.fcs.0': defaultdict(<class 'int'>, {'convolution.default'



 ## 查找FLOPS消耗最大的模块

 为了找到前向传播中FLOPS消耗最大的模块，我们编写一个递归函数

 遍历嵌套的defaultdict结构（因为模型子模块是层级嵌套的）：





In [5]:
# 导入defaultdict（用于处理嵌套的统计结果）
from collections import defaultdict


# 递归查找嵌套字典中最大的FLOPS值
# 参数说明：
#   flop_count_dict: 嵌套的FLOPS统计字典（键：模块名，值：FLOPS数或子模块字典）
#   max_value: 当前递归层级的最大FLOPS值（初始为0）
# 返回值：
#   整个嵌套字典中的最大FLOPS数值
def get_max_flops(flop_count_dict, max_value=0):
    # 遍历字典中的每个键值对（模块名：FLOPS/子模块字典）
    for _, value in flop_count_dict.items():
        # 如果当前值是整数（叶子节点，直接表示该模块的FLOPS）
        if isinstance(value, int):
            # 更新当前最大FLOPS值
            max_value = max(max_value, value)

        # 如果当前值是defaultdict（非叶子节点，包含子模块）
        elif isinstance(value, defaultdict):
            # 递归遍历子模块字典，获取子层级的最大FLOPS
            new_val = get_max_flops(value, max_value)
            # 更新全局最大FLOPS值
            max_value = max(max_value, new_val)
    # 返回当前层级及所有子层级的最大FLOPS
    return max_value


# 打印前向传播中单个模块的最大FLOPS消耗
print(f"FNO前向传播的最大单模块FLOPS消耗: {get_max_flops(fno_forward_flops)}")
# 打印反向传播中单个模块的最大FLOPS消耗
print(f"FNO反向传播的最大单模块FLOPS消耗: {get_max_flops(fno_backward_flops)}")

FNO前向传播的最大单模块FLOPS消耗: 2982150144
FNO反向传播的最大单模块FLOPS消耗: 5939134464
